In [5]:
import pandas as pd
import numpy as np

In [6]:
df = pd.read_excel('Dataset Pizahut.xlsx', sheet_name='Purchasing data')
print(df.head(50))

    BillID    Channel    OrderFrom            TransactionDate  SalesAmount  \
0        0  Take Away  CALL CENTER  2021-10-01 00:00:00+00:00       296891   
1        1  Take Away        STORE  2021-10-01 00:00:00+00:00       301782   
2        2  Take Away      WEBSITE  2021-10-01 00:00:00+00:00       319792   
3        3  Take Away        STORE  2021-10-01 00:00:00+00:00       424762   
4        4   Delivery        STORE  2021-10-01 00:00:00+00:00       280031   
5        5  Take Away        STORE  2021-10-01 00:00:00+00:00       281419   
6        6  Take Away        STORE  2021-10-01 00:00:00+00:00       191624   
7        7  Take Away        STORE  2021-10-01 00:00:00+00:00       201069   
8        8  Take Away  CALL CENTER  2021-10-01 00:00:00+00:00       301044   
9        9  Take Away        STORE  2021-10-01 00:00:00+00:00        82851   
10      10  Take Away  CALL CENTER  2021-10-01 00:00:00+00:00       157619   
11      11  Take Away        STORE  2021-10-01 00:00:00+00:00   

In [36]:
# Get data needed for analysis
df = df.loc[:, ['CustomerID', 'TransactionDate', 'SalesAmount']]

# Calculate the total revenue of each customer
df['Total Sale'] = df.groupby('CustomerID')['SalesAmount'].transform('sum')

#Take out the last order of each customer 
df['Last Order'] = df.groupby('CustomerID')['TransactionDate'].transform('max')

#Transform last order day 
df['Last Order'] = pd.to_datetime(df['Last Order'])
df['Last Order'] = pd.to_datetime(df['Last Order']).dt.strftime('%Y-%m-%d')

# Calculate the transaction date
df['TransactionDate'] = pd.to_datetime(df['TransactionDate'])
df['TransactionDate'] = df['TransactionDate'].dt.strftime('%Y-%m-%d')
df['TransactionDate'] = pd.to_datetime(df['TransactionDate']).dt.strftime('%Y-%m-%d')

# Create report date
df['ReportDate'] = '2023-01-10'
df['ReportDate'] = pd.to_datetime(df['ReportDate'])

print(df.head(50))

    CustomerID TransactionDate  SalesAmount  Total Sale  Last Order ReportDate
0      1753863      2021-10-01       296891     1506161  2022-02-06 2023-01-10
1      1124050      2021-10-01       301782     1183064  2022-01-26 2023-01-10
2      1626827      2021-10-01       319792      993004  2022-09-15 2023-01-10
3       125643      2021-10-01       424762     9923370  2022-12-09 2023-01-10
4      2117237      2021-10-01       280031     4058641  2022-10-23 2023-01-10
5      1954725      2021-10-01       281419      281419  2021-10-01 2023-01-10
6      2097093      2021-10-01       191624     2373841  2022-12-31 2023-01-10
7       602645      2021-10-01       201069      201069  2021-10-01 2023-01-10
8      1223148      2021-10-01       301044     1877960  2022-08-22 2023-01-10
9        86157      2021-10-01        82851       82851  2021-10-01 2023-01-10
10     1448797      2021-10-01       157619     1509047  2023-01-08 2023-01-10
11      433624      2021-10-01       439104      439

Calculate Recency

In [37]:
# Ensure 'Last Order' is a datetime column
df['Last Order'] = pd.to_datetime(df['Last Order'])

# Ensure 'ReportDate' is a datetime column
df['ReportDate'] = pd.to_datetime(df['ReportDate'])

# Calculate the Recency
df['Recency'] = (df['ReportDate'] - df['Last Order']).dt.days
print(df['Recency'])




0          338
1          349
2          117
3           32
4           79
          ... 
1048570      1
1048571      1
1048572      1
1048573      1
1048574      1
Name: Recency, Length: 1048575, dtype: int64


In [38]:
df['Recency'].describe()

count    1.048575e+06
mean     1.704711e+02
std      1.348822e+02
min      1.000000e+00
25%      4.700000e+01
50%      1.430000e+02
75%      2.780000e+02
max      4.660000e+02
Name: Recency, dtype: float64

In [41]:
df['R'] = df['Recency'].apply(lambda x: 1 if x <= 47 else(2 if x>143 and x<=278 else(3 if x>278 and x<=466 else 4)))
print(df['R'])

0          3
1          3
2          4
3          1
4          4
          ..
1048570    1
1048571    1
1048572    1
1048573    1
1048574    1
Name: R, Length: 1048575, dtype: int64


Calculate Frequency

In [48]:
df['Frequency'] = df.groupby('CustomerID')['TransactionDate'].rank(method='dense', ascending=True)
df['Max Frequency'] = df.groupby('CustomerID')['Frequency'].transform('max')
df['Frequency'] = pd.to_numeric(df['Frequency']).astype('int')

In [49]:
df['Frequency'].describe()

count    1.048575e+06
mean     2.746682e+00
std      4.436661e+00
min      1.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      3.000000e+00
max      1.810000e+02
Name: Frequency, dtype: float64

In [51]:
max_frequency = df['Frequency'].max()
print(max_frequency)
#181 maybe the outlier so we define max frequency is 4

181


In [70]:
df['F'] = df['Frequency'].apply(lambda x: 1 if x <= 2 else(2 if x>2 and x<=3 else(3 if x>3 and x<=181 else 4)))

In [53]:
print(df['F'])

0          1
1          1
2          1
3          1
4          1
          ..
1048570    1
1048571    4
1048572    2
1048573    1
1048574    4
Name: F, Length: 1048575, dtype: int64


Calculate the Moneytary


In [69]:
print(df['Total Sale'])

0          1506161
1          1183064
2           993004
3          9923370
4          4058641
            ...   
1048570     178107
1048571    1889312
1048572    1119060
1048573     797890
1048574    3113654
Name: Total Sale, Length: 1048575, dtype: int64


In [68]:
df['Total Sale'].describe()

count    1.048575e+06
mean     1.357587e+06
std      2.521858e+06
min      7.148000e+03
25%      2.790525e+05
50%      6.194030e+05
75%      1.548363e+06
max      1.337745e+08
Name: Total Sale, dtype: float64

In [71]:
df['M'] = df['Total Sale'].apply(lambda x: 1 if x <= 7148 else(2 if x> 279052 and x<= 619403 else(3 if x>619403 and x<=1548363 else 4)))

In [72]:
print([df['M']])

[0          3
1          3
2          3
3          4
4          4
          ..
1048570    4
1048571    4
1048572    3
1048573    3
1048574    4
Name: M, Length: 1048575, dtype: int64]


Calculate RFM


In [73]:
df['RFM'] = df['R'].astype(str) + df['F'].astype(str) + df['M'].astype(str)


In [74]:
df

,CustomerID,TransactionDate,SalesAmount,Total Sale,Last Order,ReportDate,Recency,R,Frequency,Max Frequency,F,M,RFM
0,1753863,2021-10-01,296891,1506161,2022-02-06,2023-01-10,338,3,1,3.0,1,3,313
1,1124050,2021-10-01,301782,1183064,2022-01-26,2023-01-10,349,3,1,3.0,1,3,313
2,1626827,2021-10-01,319792,993004,2022-09-15,2023-01-10,117,4,1,5.0,1,3,413
3,125643,2021-10-01,424762,9923370,2022-12-09,2023-01-10,32,1,1,47.0,1,4,114
4,2117237,2021-10-01,280031,4058641,2022-10-23,2023-01-10,79,4,1,16.0,1,4,414
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,1398136,2023-01-09,178107,178107,2023-01-09,2023-01-10,1,1,1,1.0,1,4,114
1048571,2023258,2023-01-09,331447,1889312,2023-01-09,2023-01-10,1,1,8,8.0,3,4,134
1048572,2041510,2023-01-09,381509,1119060,2023-01-09,2023-01-10,1,1,3,3.0,2,3,123
1048573,1927215,2023-01-09,508205,797890,2023-01-09,2023-01-10,1,1,2,2.0,1,3,113
